In [1]:
import csv
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_xls = "https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide"
url_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
url_countries_csv = "https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv"

In [3]:
page = requests.get(url_xls).text

In [4]:
soup = BeautifulSoup(page, 'html.parser')

In [5]:
data_attribute = soup.find_all("a", attrs={"data-toggle": "tooltip"})

In [6]:
for link in data_attribute:
    data_link = link.get('href')

In [7]:
with requests.Session() as s:
    download = s.get(data_link).content
    dataframe = pd.read_excel(download)

In [8]:
dataframe.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-22,22,3,2020,0,0,Afghanistan,AF
1,2020-03-21,21,3,2020,2,0,Afghanistan,AF
2,2020-03-20,20,3,2020,0,0,Afghanistan,AF
3,2020-03-19,19,3,2020,0,0,Afghanistan,AF
4,2020-03-18,18,3,2020,1,0,Afghanistan,AF


In [9]:
dataframe.groupby("Countries and territories").mean().head()

,Day,Month,Year,Cases,Deaths
Countries and territories,,,,,
Afghanistan,15.506849,1.876712,2019.986301,0.328767,0.000000
Albania,15.500000,3.000000,2020.000000,5.428571,0.142857
Algeria,15.076923,1.948718,2019.987179,1.205128,0.128205
Andorra,16.666667,3.000000,2020.000000,9.777778,0.000000
Angola,22.000000,3.000000,2020.000000,2.000000,0.000000


In [10]:
df_github = pd.read_csv(url_csv)

In [11]:
data_by_country = df_github.groupby("Country/Region").mean()

In [12]:
df_github.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,70,75,82,114,147,177,212,272,322,411
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,639,701,773,839,825,878,889,924,963,1007
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,178,200,212,226,243,266,313,345,385,432
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,149,197,238,428,566,673,790,900,1030,1183


In [13]:
df_github_countries = df_github.iloc[:, 0:4]

In [14]:
df_github_countries

,Province/State,Country/Region,Lat,Long
0,NaN,Thailand,15.0000,101.0000
1,NaN,Japan,36.0000,138.0000
2,NaN,Singapore,1.2833,103.8333
3,NaN,Nepal,28.1667,84.2500
4,NaN,Malaysia,2.5000,112.5000
...,...,...,...,...
477,Northwest Territories,Canada,64.8255,-124.8457
478,NaN,Cape Verde,15.1111,-23.6167
479,NaN,East Timor,-8.5500,125.5600
480,NaN,Eritrea,15.1794,39.7823


In [15]:
df_github_countries.iloc[0,0]

nan

In [92]:
country = df_github_countries.iloc[:, 1:4].drop_duplicates(subset=['Country/Region'],keep='last',ignore_index=True)

In [93]:
len(country)

166

In [18]:
country.to_csv("country.csv")

In [80]:
continents = ["Africa", "Antarctica", "Asia", "Europe", "North America", "Oceania", "South America", "None"]

In [81]:
continents_df = pd.DataFrame(continents)

In [82]:
continents_df.to_csv("continent.csv")

In [22]:
prov_state = df_github.iloc[:, 0].drop_duplicates(keep='first')

In [23]:
prov_state = prov_state.reset_index()

In [24]:
prov_state = prov_state.drop(["index"],axis=1)

In [25]:
prov_state.iloc[0, :] = "No province/state"

In [26]:
prov_state

,Province/State
0,No province/state
1,British Columbia
2,New South Wales
3,Victoria
4,Queensland
...,...
319,New Caledonia
320,Bermuda
321,Sint Maarten
322,Isle of Man


In [27]:
provstate_noquotes = []
for provstate in prov_state.iloc[:,0]:
    provstate_noquotes.append(str(provstate).replace('"', ''))

In [28]:
prov_state['Province/State_noquotes'] = provstate_noquotes

In [29]:
prov_state=prov_state.drop("Province/State",axis=1)

In [30]:
prov_state.to_csv("province_state.csv")

In [31]:
import math
region_codes = []
for region in df_github_countries.iloc[:,0]:
    if isinstance(region, str):
        pass
    elif np.isnan(region):
        region_codes.append(322)
        continue
    for i, item in enumerate(prov_state.iloc[:,0]):
        if item == region:
            region_codes.append(i)

print(len(region_codes))

482


In [32]:
df_github_countries["region_codes"] = region_codes

In [33]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes
0,NaN,Thailand,15.0000,101.0000,322
1,NaN,Japan,36.0000,138.0000,322
2,NaN,Singapore,1.2833,103.8333,322
3,NaN,Nepal,28.1667,84.2500,322
4,NaN,Malaysia,2.5000,112.5000,322
...,...,...,...,...,...
477,Northwest Territories,Canada,64.8255,-124.8457,323
478,NaN,Cape Verde,15.1111,-23.6167,322
479,NaN,East Timor,-8.5500,125.5600,322
480,NaN,Eritrea,15.1794,39.7823,322


In [34]:
df_countries_continents = pd.read_csv(url_countries_csv)

In [35]:
df_countries_continents

,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
189,South America,Paraguay
190,South America,Peru
191,South America,Suriname
192,South America,Uruguay


In [41]:
continent_codes = []
for item in df_github_countries.iloc[:,1]:
    old_size = len(continent_codes)
    for country in df_countries_continents.iterrows():
        if item==country[1][1]:
            continent_codes.append(country[1][0])
            
    new_size = len(continent_codes)
    if old_size==new_size:
        continent_codes.append("NA")

In [42]:
df_github_countries['continent'] = continent_codes

In [43]:
df_github_countries.head()

,Province/State,Country/Region,Lat,Long,region_codes,continent
0,NaN,Thailand,15.0000,101.0000,322,Asia
1,NaN,Japan,36.0000,138.0000,322,Asia
2,NaN,Singapore,1.2833,103.8333,322,Asia
3,NaN,Nepal,28.1667,84.2500,322,Asia
4,NaN,Malaysia,2.5000,112.5000,322,Asia


In [60]:
df_github_countries.iloc[1,1]

'Japan'

In [74]:
for i, line in enumerate(df_github_countries.iterrows()):
    if line [1][1]=="North Macedonia":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Martinique":
        df_github_countries.iloc[i,5]="North America"
    elif line[1][1]=="Burkina Faso":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Holy See":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Cruise Ship":
        df_github_countries.iloc[i,5]="None"
    elif line[1][1]=="Czechia":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Taiwan*":
        df_github_countries.iloc[i,5]="Asia"
    elif line[1][1]=="Russia":
        df_github_countries.iloc[i,5]="Asia"
    elif line[1][1]=="Congo (Kinshasa)":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Cote d'Ivoire":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Eswatini":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Kosovo":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Congo (Brazzaville)":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Gambia, The":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Bahamas, The":
        df_github_countries.iloc[i,5]="North America"
    elif line[1][1]=="Cabo Verde":
        df_github_countries.iloc[i,5]="Africa"

In [75]:
for i, line in enumerate(df_github_countries.iterrows()):
    if line[1][5]=="NA":
        print(line)

In [76]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent
0,NaN,Thailand,15.0000,101.0000,322,Asia
1,NaN,Japan,36.0000,138.0000,322,Asia
2,NaN,Singapore,1.2833,103.8333,322,Asia
3,NaN,Nepal,28.1667,84.2500,322,Asia
4,NaN,Malaysia,2.5000,112.5000,322,Asia
...,...,...,...,...,...,...
477,Northwest Territories,Canada,64.8255,-124.8457,323,North America
478,NaN,Cape Verde,15.1111,-23.6167,322,Africa
479,NaN,East Timor,-8.5500,125.5600,322,Asia
480,NaN,Eritrea,15.1794,39.7823,322,Africa


In [83]:
import math
continent_codes = []
for continent in df_github_countries.iloc[:,-1]:
    if isinstance(continent, str):
        pass
    elif np.isnan(continent):
        print("WHAT")
    for i, item in enumerate(continents_df.iloc[:,0]):
        if item == continent:
            continent_codes.append(i)

print(len(continent_codes))

482


In [85]:
df_github_countries['continent_codes'] = continent_codes

In [86]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
477,Northwest Territories,Canada,64.8255,-124.8457,323,North America,4
478,NaN,Cape Verde,15.1111,-23.6167,322,Africa,0
479,NaN,East Timor,-8.5500,125.5600,322,Asia,2
480,NaN,Eritrea,15.1794,39.7823,322,Africa,0


In [102]:
cases = df_github_countries.drop("continent", axis=1).drop("Lat", axis=1).drop("Long", axis=1).drop("Province/State", axis=1)

In [103]:
cases

,Country/Region,region_codes,continent_codes
0,Thailand,322,2
1,Japan,322,2
2,Singapore,322,2
3,Nepal,322,2
4,Malaysia,322,2
...,...,...,...
477,Canada,323,4
478,Cape Verde,322,0
479,East Timor,322,2
480,Eritrea,322,0


In [104]:
import math
country_codes = []
for case in cases.iloc[:,0]:
    for i, item in enumerate(country.iloc[:,0]):
        if item == case:
            country_codes.append(i)
            
print(len(country_codes))

482


In [105]:
cases['country_code'] = country_codes

In [106]:
cases = cases.drop('Country/Region', axis=1)

In [107]:
cases

,region_codes,continent_codes,country_code
0,322,2,0
1,322,2,1
2,322,2,2
3,322,2,3
4,322,2,4
...,...,...,...
477,323,4,161
478,322,0,162
479,322,2,163
480,322,0,164


In [108]:
cases_full = pd.concat([cases, df_github.iloc[:,4:]], axis=1)

In [124]:
cases_full_melt = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=['1/22/20'])

In [125]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2
1,322,2,1,1/22/20,2
2,322,2,2,1/22/20,0
3,322,2,3,1/22/20,0
4,322,2,4,1/22/20,0
...,...,...,...,...,...
477,323,4,161,1/22/20,0
478,322,0,162,1/22/20,0
479,322,2,163,1/22/20,0
480,322,0,164,1/22/20,0


In [117]:
cases_full.shape

(482, 63)

In [126]:
for i, column in enumerate(cases_full.iloc[:,4:]):
    melted_df = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=[column])
    cases_full_melt = cases_full_melt.append(melted_df)
    print(i, cases_full_melt.shape)

0 (964, 5)
1 (1446, 5)
2 (1928, 5)
3 (2410, 5)
4 (2892, 5)
5 (3374, 5)
6 (3856, 5)
7 (4338, 5)
8 (4820, 5)
9 (5302, 5)
10 (5784, 5)
11 (6266, 5)
12 (6748, 5)
13 (7230, 5)
14 (7712, 5)
15 (8194, 5)
16 (8676, 5)
17 (9158, 5)
18 (9640, 5)
19 (10122, 5)
20 (10604, 5)
21 (11086, 5)
22 (11568, 5)
23 (12050, 5)
24 (12532, 5)
25 (13014, 5)
26 (13496, 5)
27 (13978, 5)
28 (14460, 5)
29 (14942, 5)
30 (15424, 5)
31 (15906, 5)
32 (16388, 5)
33 (16870, 5)
34 (17352, 5)
35 (17834, 5)
36 (18316, 5)
37 (18798, 5)
38 (19280, 5)
39 (19762, 5)
40 (20244, 5)
41 (20726, 5)
42 (21208, 5)
43 (21690, 5)
44 (22172, 5)
45 (22654, 5)
46 (23136, 5)
47 (23618, 5)
48 (24100, 5)
49 (24582, 5)
50 (25064, 5)
51 (25546, 5)
52 (26028, 5)
53 (26510, 5)
54 (26992, 5)
55 (27474, 5)
56 (27956, 5)
57 (28438, 5)
58 (28920, 5)


In [120]:
cases_full_melt.shape

(28920, 5)

In [127]:
60*482

28920

In [129]:
cases_full_melt = cases_full_melt.reset_index()

In [134]:
cases_full_melt = cases_full_melt.drop("index", axis=1)

In [135]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2
1,322,2,1,1/22/20,2
2,322,2,2,1/22/20,0
3,322,2,3,1/22/20,0
4,322,2,4,1/22/20,0
...,...,...,...,...,...
28915,323,4,161,3/21/20,1
28916,322,0,162,3/21/20,1
28917,322,2,163,3/21/20,1
28918,322,0,164,3/21/20,1


In [136]:
cases_full_melt['variable'] = pd.to_datetime(cases_full_melt['variable'],infer_datetime_format=True)

In [137]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,2020-01-22,2
1,322,2,1,2020-01-22,2
2,322,2,2,2020-01-22,0
3,322,2,3,2020-01-22,0
4,322,2,4,2020-01-22,0
...,...,...,...,...,...
28915,323,4,161,2020-03-21,1
28916,322,0,162,2020-03-21,1
28917,322,2,163,2020-03-21,1
28918,322,0,164,2020-03-21,1


In [140]:
cases_full_melt.to_csv('cases_confirmed.csv')